In [1]:
from huggingface_hub import login
login()   # follows the prompt to paste your HF token


In [2]:
!pip install --upgrade datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 4.2 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cufft-cu12 11.4.0.6 which is incompatible.
torch 2.6.0+

In [3]:
# Remove the old cache just in case (optional, but good practice if you switch methods)
!rm -rf /content/hf_cache
!rm -rf ~/.cache/huggingface/datasets/nico8771___swda_processed # Clear default cache too if needed

from datasets import load_dataset
import os

# Just load the dataset without specifying cache_dir
dataset = load_dataset(
    "nico8771/mrda_processed"
    # No cache_dir argument here
)

print(dataset)

README.md:   0%|          | 0.00/837 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.75M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/392k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/377k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/75067 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/16433 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/16702 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['caller', 'text', 'act_tag', 'conversation_no', 'speaker_change'],
        num_rows: 75067
    })
    validation: Dataset({
        features: ['caller', 'text', 'act_tag', 'conversation_no', 'speaker_change'],
        num_rows: 16433
    })
    test: Dataset({
        features: ['caller', 'text', 'act_tag', 'conversation_no', 'speaker_change'],
        num_rows: 16702
    })
})


In [4]:
from collections import Counter
import pandas as pd
import torch

# ------------------------------------------------------------------
# 1.  Per-split class statistics
# ------------------------------------------------------------------
def split_stats(ds_dict, label_key="act_tag"):
    rows = []
    all_counter = Counter()
    for split, ds in ds_dict.items():
        counter = Counter(ds[label_key])
        total = sum(counter.values())
        for lbl, cnt in counter.items():
            rows.append({"split": split, "label": lbl,
                         "count": cnt,
                         "pct": 100.0 * cnt / total})
        all_counter.update(counter)

    # Global totals (optional)
    grand_total = sum(all_counter.values())
    for lbl, cnt in all_counter.items():
        rows.append({"split": "ALL", "label": lbl,
                     "count": cnt,
                     "pct": 100.0 * cnt / grand_total})

    return pd.DataFrame(rows)

stats_df = split_stats(dataset)
print(stats_df.head())          # sanity check

# ------------------------------------------------------------------
# 2.  Inverse-√freq class-weight tensor
# ------------------------------------------------------------------
label_list = sorted({row["label"] for row in stats_df.to_dict("records")})
freq = torch.tensor(
    [stats_df[(stats_df.label == lbl) & (stats_df.split == "ALL")]["count"].values[0]
     for lbl in label_list],
    dtype=torch.float,
)
class_weights = 1.0 / torch.sqrt(freq)
class_weights /= class_weights.mean()          # (optional) re-normalize
print(class_weights)                           # plug into CrossEntropyLoss

   split  label  count        pct
0  train      2   5324   7.092331
1  train      4  45099  60.078330
2  train      1   9739  12.973743
3  train      0  10265  13.674451
4  train      3   4640   6.181145
tensor([0.9445, 0.9468, 1.2916, 1.3666, 0.4506])


In [5]:
from collections import defaultdict
from datasets import Dataset, DatasetDict
from tqdm.auto import tqdm

CHUNK_SIZE = 175

# 1) Build a conversation‐level Dataset: one row per conversation
def make_conversation_dataset(split_ds):
    conv_accum = defaultdict(lambda: {
        "caller": [],
        "text": [],
        "act_tag": [],
        "speaker_change": [],
    })
    for row in tqdm(
        split_ds,
        desc="Grouping into conversations",
        leave=False
    ):
        cid = row["conversation_no"]
        conv_accum[cid]["caller"].append(row["caller"])
        conv_accum[cid]["text"].append(row["text"])
        conv_accum[cid]["act_tag"].append(row["act_tag"])
        conv_accum[cid]["speaker_change"].append(row["speaker_change"])
    # flatten into a list of dicts
    conv_list = []
    for cid, fields in conv_accum.items():
        fields["conversation_no"] = cid
        conv_list.append(fields)
    return Dataset.from_list(conv_list)

# 2) Chunk each conversation‐row into windows of up to CHUNK_SIZE
def chunk_conversation(batch):
    all_texts          = batch["text"][0]
    all_callers        = batch["caller"][0]
    all_act_tags       = batch["act_tag"][0]
    all_speaker_change = batch["speaker_change"][0]
    cid                = batch["conversation_no"][0]

    out = {
        "text": [], "caller": [], "act_tag": [],
        "speaker_change": [], "conversation_no": [],
    }

    for start in range(0, len(all_texts), CHUNK_SIZE):
        end = start + CHUNK_SIZE
        out["text"].append(all_texts[start:end])
        out["caller"].append(all_callers[start:end])
        out["act_tag"].append(all_act_tags[start:end])
        out["speaker_change"].append(all_speaker_change[start:end])
        out["conversation_no"].append(cid)

    return out

# 3) Apply to each split
raw = dataset  # your original DatasetDict

# First turn utterances → conversations
conv_ds = {}
for split in ["train", "validation", "test"]:
    conv_ds[split] = make_conversation_dataset(raw[split])

# Then chunk each conversation
chunked = DatasetDict()
for split in ["train", "validation", "test"]:
    chunked[split] = (
        conv_ds[split]
        .map(
            chunk_conversation,
            batched=True,
            batch_size=1,
            num_proc=4,
            remove_columns=conv_ds[split].column_names,
            desc=f"Chunking {split}"
        )
    )


Grouping into conversations:   0%|          | 0/75067 [00:00<?, ?it/s]

Grouping into conversations:   0%|          | 0/16433 [00:00<?, ?it/s]

Grouping into conversations:   0%|          | 0/16702 [00:00<?, ?it/s]

Chunking train (num_proc=4):   0%|          | 0/51 [00:00<?, ? examples/s]

Chunking validation (num_proc=4):   0%|          | 0/12 [00:00<?, ? examples/s]

Chunking test (num_proc=4):   0%|          | 0/12 [00:00<?, ? examples/s]

In [6]:
print(chunked)

print(chunked["train"][0])

DatasetDict({
    train: Dataset({
        features: ['caller', 'text', 'act_tag', 'speaker_change', 'conversation_no'],
        num_rows: 453
    })
    validation: Dataset({
        features: ['caller', 'text', 'act_tag', 'speaker_change', 'conversation_no'],
        num_rows: 101
    })
    test: Dataset({
        features: ['caller', 'text', 'act_tag', 'speaker_change', 'conversation_no'],
        num_rows: 102
    })
})
{'caller': ['fe016', 'fe016', 'fe016', 'fe016', 'fe016', 'fe016', 'fe016', 'fe016', 'fe016', 'fe016', 'me011', 'me011', 'me011', 'fe016', 'fe016', 'me018', 'me011', 'me011', 'me011', 'fe016', 'fe016', 'fe016', 'me011', 'fe016', 'me011', 'me011', 'fe016', 'fe016', 'fe016', 'me011', 'me011', 'fe016', 'me011', 'me011', 'me011', 'me011', 'me011', 'me011', 'me011', 'me011', 'me011', 'me011', 'mn017', 'me011', 'fe016', 'fe016', 'mn017', 'me011', 'me011', 'me011', 'me011', 'me011', 'me011', 'fe016', 'me011', 'me011', 'me011', 'me011', 'me011', 'me011', 'me011', 'me011', '

In [7]:
import torch
from transformers import AutoTokenizer

tokenizer    = AutoTokenizer.from_pretrained("roberta-base")
PAD_LABEL_ID = -100  # your label-padding ID


def collate_fn(batch):
    bs = len(batch)

    # 1) Utterance‐level pad up to CHUNK_SIZE
    padded_texts, padded_speaker, padded_labels, utt_masks = [], [], [], []
    for ex in batch:
        texts = ex["text"]
        spch  = ex["speaker_change"]
        tags  = ex["act_tag"]
        n = len(texts)
        pad_n = CHUNK_SIZE - n

        padded_texts.append(texts + [""] * pad_n)
        padded_speaker.append(spch  + [0] * pad_n)
        padded_labels.append(tags  + [PAD_LABEL_ID] * pad_n)
        utt_masks.append([1] * n + [0] * pad_n)  # 1=real, 0=padded

    # 2) Flatten all utterances into one list (bs * CHUNK_SIZE)
    flat_utts = sum(padded_texts, [])

    # 3) Tokenize in one batch
    enc       = tokenizer(
        flat_utts,
        padding="longest",
        truncation=True,
        return_tensors="pt"
    )
    flat_ids  = enc["input_ids"]       # [bs*CHUNK_SIZE, seq_len]
    flat_mask = enc["attention_mask"]  # [bs*CHUNK_SIZE, seq_len]

    # 4) Reshape back to [bs, CHUNK_SIZE, seq_len]
    seq_len        = flat_ids.size(-1)
    input_ids      = flat_ids.view(bs, CHUNK_SIZE, seq_len)
    attention_mask = flat_mask.view(bs, CHUNK_SIZE, seq_len)

    # 5) Build speaker_change, labels, and utt_mask tensors
    speaker_change = torch.tensor(padded_speaker, dtype=torch.long)     # [bs, CHUNK_SIZE]
    labels         = torch.tensor(padded_labels,  dtype=torch.long)     # [bs, CHUNK_SIZE]
    utt_mask       = torch.tensor(utt_masks,      dtype=torch.long)     # [bs, CHUNK_SIZE]

    return {
        "input_ids":      input_ids,
        "attention_mask": attention_mask,
        "speaker_change": speaker_change,
        "utt_mask":       utt_mask,
        "labels":         labels,
    }


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [8]:
from torch.utils.data import DataLoader



train_loader = DataLoader(
    chunked["train"],
    batch_size=1,          # or whatever fits your GPU
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=2           # to parallelize I/O
)

valid_loader = DataLoader(
    chunked["validation"],
    batch_size=1,            # or whatever you choose
    shuffle=False,           # no need to shuffle validation
    collate_fn=collate_fn,
    num_workers=2
)


In [9]:
import torch
import torch.nn as nn
from transformers import AutoModel
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


class DASeq2SeqClassifier(nn.Module):
    def __init__(self, num_labels, hidden_size=256, pad_label_id=-100, bos_label_id=0):
        """
        num_labels: number of DA tags
        hidden_size: GRU hidden size per direction
        pad_label_id: label ID used for padding (will be ignored in loss)
        """
        super().__init__()
        # 1) Encoder backbone
        self.roberta = AutoModel.from_pretrained("roberta-base")
        enc_dim = self.roberta.config.hidden_size  # typically 768

        # 2) Speaker‐turn embeddings (optional, 2 classes: change/no_change)
        self.turn_emb = nn.Embedding(2, enc_dim)

        # 3) Bi‐GRU encoder over utterance‐level embeddings
        self.encoder_gru = nn.GRU(
            input_size=enc_dim,
            hidden_size=hidden_size,
            num_layers=1,
            bidirectional=True,
            batch_first=True
        )

        # 4) Project Bi‐GRU → hidden_size
        self.ctx_proj    = nn.Linear(hidden_size*2, hidden_size)


        # 5) Final classifier to produce logits over DA tags
        self.classifier = nn.Linear(hidden_size, num_labels)

        # 6) Loss (ignore padding positions)
        self.loss_fct = nn.CrossEntropyLoss(weight=class_weights, ignore_index=pad_label_id)

        # store IDs
        self.pad_label_id = pad_label_id

    def forward(self, input_ids, attention_mask, speaker_change, utt_mask, labels=None):
        """
        input_ids:       [batch, chunk_size, seq_len]
        attention_mask:  [batch, chunk_size, seq_len]
        speaker_change:  [batch, chunk_size] ints 0/1
        labels:          [batch, chunk_size] ints in [0..num_labels) or pad_label_id
        """
        bs, cs, sl = input_ids.size()

        # 1) Flatten to feed into RoBERTa
        flat_ids   = input_ids.view(bs*cs, sl)
        flat_mask  = attention_mask.view(bs*cs, sl)
        roberta_out = self.roberta(flat_ids, attention_mask=flat_mask).last_hidden_state
        cls_flat    = roberta_out[:, 0, :]        # [bs*cs, enc_dim]

        # 2) Reshape back to [batch, chunk, enc_dim]
        utt_emb = cls_flat.view(bs, cs, -1)

        # 3) Add speaker‐turn embeddings
        turn_emb = self.turn_emb(speaker_change)  # [bs, cs, enc_dim]
        utt_emb  = utt_emb + turn_emb

        # 4) Encode utterance sequence with Bi‐GRU, skipping padded time‐steps
        #    utt_mask: 1 for real, 0 for pad, shape [bs, cs]
        lengths = utt_mask.sum(dim=1).cpu()       # [bs], on CPU for pack
        packed  = torch.nn.utils.rnn.pack_padded_sequence(
            utt_emb, lengths, batch_first=True, enforce_sorted=False
        )
        packed_out, _ = self.encoder_gru(packed)
        enc_out, _    = torch.nn.utils.rnn.pad_packed_sequence(
            packed_out, batch_first=True, total_length=utt_emb.size(1)
        )                                         # [bs, cs, hidden*2]

        # 5) Split into forward/backward halves
        fwd, bwd = torch.chunk(enc_out, 2, dim=-1)  # each [bs, cs, hidden]

        # 4) Project to hidden_size
        ctx = self.ctx_proj(enc_out)  # [bs, cs, hidden]

        # 5) Classify each utterance
        logits = self.classifier(ctx)  # [bs, cs, num_labels]

        output = {"logits": logits}

        # 6) Compute token/utterance-level cross-entropy loss (ignore pad_label_id)
        if labels is not None:
            loss = self.loss_fct(
                logits.view(-1, logits.size(-1)),  # [bs*cs, num_labels]
                labels.view(-1)                     # [bs*cs]
            )
            output["loss"] = loss

        return output


In [ ]:
import os
import time
import torch
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
from transformers import AutoConfig
from huggingface_hub import hf_hub_download, upload_file
from torch.nn.utils import clip_grad_norm_

# ── 1) Settings ─────────────────────────────────────────────────────────────────
repo_id    = "nico8771/mrda-bigruberta-nodec-lr-hybrid"
save_dir   = "./da_model"
os.makedirs(save_dir, exist_ok=True)

# the name of the checkpoint you expect on the Hub
ckpt_fname = "checkpoint_epoch07.pt"

device     = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 1
lr         = 1e-4
weight_decay=5e-5
max_grad_norm = 5.0

# ── 2) Download & load existing checkpoint (if any) ──────────────────────────────
try:
    ckpt_path = hf_hub_download(
        repo_id=repo_id,
        filename=ckpt_fname,
        repo_type="model",
        revision="main",
    )
    ckpt = torch.load(ckpt_path, map_location="cpu")
    start_epoch = ckpt["epoch"] + 1
    print(f"Resuming from epoch {start_epoch}")
except Exception:
    # no checkpoint on HF Hub: start from scratch
    ckpt = None
    start_epoch = 1
    print("No existing checkpoint found; starting from epoch 1")

# ── 3) Rebuild & load your model ─────────────────────────────────────────────────

model  = DASeq2SeqClassifier(
    num_labels=5,
    hidden_size=256,
)

if ckpt is not None:
    model.load_state_dict(ckpt["model_state"])
model.to(device)

# ── 4) Freeze RoBERTa + turn embeddings ─────────────────────────────────────────
# Unfreeze the embeddings only if you want (typically keep them frozen longer)
model.roberta.embeddings.requires_grad_(False)
model.turn_emb.weight.requires_grad = False

# Total number of encoder layers
total_layers = len(model.roberta.encoder.layer) #total layers = 12
n_layers_to_unfreeze = 12

for p in model.roberta.parameters():
    p.requires_grad = True #true iff all unfrezed

if n_layers_to_unfreeze > 0:
    for i in range(total_layers - n_layers_to_unfreeze, total_layers):
        for param in model.roberta.encoder.layer[i].parameters():
            param.requires_grad = True

trainable = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(trainable, lr=lr, weight_decay=weight_decay)

# ── 5) (Re)load optimizer state if checkpoint existed ───────────────────────────
# if ckpt is not None and "optim_state" in ckpt:
#     optimizer.load_state_dict(ckpt["optim_state"])

# ── 6) Training + Validation loops ──────────────────────────────────────────────
for epoch in range(start_epoch, start_epoch + num_epochs):
    # ---- Train ----
    model.train()
    train_losses = []
    t0 = time.time()
    for batch in tqdm(train_loader, desc=f"[Epoch {epoch}] Train"):
        batch = {k: v.to(device) for k, v in batch.items() if torch.is_tensor(v)}
        optimizer.zero_grad()
        out  = model(
            batch["input_ids"],
            batch["attention_mask"],
            batch["speaker_change"],
            batch["utt_mask"],
            labels=batch["labels"],
        )
        loss = out["loss"]
        loss.backward()
        clip_grad_norm_(trainable, max_grad_norm)
        optimizer.step()
        train_losses.append(loss.item())
    avg_train_loss = sum(train_losses) / len(train_losses)
    train_time = time.time() - t0

    # ---- Validate ----
    model.eval()
    val_losses, all_preds, all_labels = [], [], []
    with torch.no_grad():
        for batch in tqdm(valid_loader, desc=f"[Epoch {epoch}] Eval"):
            batch = {k: v.to(device) for k, v in batch.items() if torch.is_tensor(v)}
            out = model(
                batch["input_ids"],
                batch["attention_mask"],
                batch["speaker_change"],
                batch["utt_mask"],
                labels=batch["labels"],
            )
            val_losses.append(out["loss"].item())

            # simple greedy decode for accuracy
            logits = out["logits"]                     # [bs, cs, num_labels]
            preds  = logits.argmax(-1).view(-1).cpu().tolist()
            labels = batch["labels"].view(-1).cpu().tolist()
            for p, l in zip(preds, labels):
                if l != model.pad_label_id:
                    all_preds.append(p)
                    all_labels.append(l)

    avg_val_loss = sum(val_losses) / len(val_losses)
    val_acc      = accuracy_score(all_labels, all_preds)

    print(
      f"Epoch {epoch:02d} • "
      f"time {train_time:.1f}s • "
      f"train_loss {avg_train_loss:.4f} • "
      f"val_loss   {avg_val_loss:.4f} • "
      f"val_acc    {val_acc:.4f}"
    )

    # ── 7) Save & upload this epoch’s checkpoint ─────────────────────────────────
    new_ckpt = {
      "epoch":       epoch,
      "model_state": model.state_dict(),
      "optim_state": optimizer.state_dict(),
    }
    new_fname = f"checkpoint_epoch{epoch:02d}.pt"
    local_path = os.path.join(save_dir, new_fname)
    torch.save(new_ckpt, local_path)

    upload_file(
      path_or_fileobj=local_path,
      path_in_repo=new_fname,
      repo_id=repo_id,
      repo_type="model",
      commit_message=(
        f"Ep {epoch:02d}: "
        f"tr_loss {avg_train_loss:.4f}, "
        f"val_loss {avg_val_loss:.4f}, "
        f"val_acc {val_acc:.4f},"
        f"Roberta {12-n_layers_to_unfreeze}/12 Frozen,"
        f"train time {train_time:.1f}s,"
      ),
      token=True
    )
    print(f"✅ Uploaded checkpoint {new_fname}")


checkpoint_epoch07.pt:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

Resuming from epoch 8


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[Epoch 8] Train:   0%|          | 0/453 [00:00<?, ?it/s]

[Epoch 8] Eval:   0%|          | 0/101 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7a731423e0c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7a731423e0c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 16

Epoch 08 • time 494.4s • train_loss 0.3041 • val_loss   0.3911 • val_acc    0.8721


checkpoint_epoch08.pt:   0%|          | 0.00/1.51G [00:00<?, ?B/s]